In [1]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report, f1_score, precision_score, accuracy_score, recall_score
import seaborn as sns
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers
import tensorflow.keras.backend as K
# Dataset preparation
train_dir = "/kaggle/input/deepfake-and-real-images/Dataset/Train"
test_dir = "/kaggle/input/deepfake-and-real-images/Dataset/Test"
val_dir = "/kaggle/input/deepfake-and-real-images/Dataset/Validation"
img_height, img_width = 256, 256
num_classes = 2
input_shape = (img_height, img_width, 3)
BATCH_SIZE = 32 

train_ds = image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    seed=69,
    image_size=(img_height, img_width),
    batch_size=BATCH_SIZE,
)

val_ds = image_dataset_from_directory(
    val_dir,
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    seed=69,
    image_size=(img_height, img_width),
    batch_size=BATCH_SIZE,
)

test_ds = image_dataset_from_directory(
    test_dir,
    image_size=(img_height, img_width),
    batch_size=BATCH_SIZE,
    labels='inferred',
    label_mode='categorical'
)



# Custom Positional Embedding Layer
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.sequence_length = sequence_length
        self.output_dim = output_dim
        self.pos_embedding = self.add_weight(
            shape=(1, sequence_length, output_dim),
            initializer='random_normal',
            trainable=True,
            name='pos_embedding'
        )

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        return inputs + tf.tile(self.pos_embedding, [batch_size, 1, 1])

def mbconv(x, expand_ratio, output_dim, stride=1):
    input_dim = x.shape[-1]
    hidden_dim = input_dim * expand_ratio
    
    residual = x
    
    # Expansion
    x = layers.Conv2D(hidden_dim, 1, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('swish')(x)
    
    # Depthwise
    x = layers.DepthwiseConv2D(3, stride, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('swish')(x)
    
    # SE
    se = layers.GlobalAveragePooling2D()(x)
    se = layers.Dense(hidden_dim // 4)(se)
    se = layers.Activation('swish')(se)
    se = layers.Dense(hidden_dim, activation='sigmoid')(se)
    se = layers.Reshape((1, 1, hidden_dim))(se)
    x = layers.Multiply()([x, se])
    
    # Output
    x = layers.Conv2D(output_dim, 1, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    
    if stride == 1 and input_dim == output_dim:
        x = layers.Add()([x, residual])
        
    return x

def transformer_block(x, dim, num_heads=4):
    shortcut = x
    
    # Multi-head self attention
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    attention_output = layers.MultiHeadAttention(
        num_heads=num_heads,
        key_dim=dim // num_heads
    )(x, x)
    x = layers.Add()([shortcut, attention_output])
    
    # FFN
    ffn_shortcut = x
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    x = layers.Dense(dim * 4, activation='gelu')(x)
    x = layers.Dense(dim)(x)
    x = layers.Add()([ffn_shortcut, x])
    
    return x

def create_proto_1():
    inputs = keras.Input(shape=input_shape)
    #x = data_augmentation(inputs)
    
    # Stage 1: Stem
    x = layers.Conv2D(64, 3, 2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('swish')(x)
    
    # Stage 2: MBConv
    x = mbconv(x, 4, 96, stride=2)
    x = mbconv(x, 4, 96, stride=1)
    
    # Stage 3: MBConv
    x = mbconv(x, 4, 192, stride=2)
    x = mbconv(x, 4, 192, stride=1)
    
    # Stage 4: Transform to sequence for transformer
    x = layers.Conv2D(384, 1)(x)
    
    # Calculate sequence length
    h = input_shape[0] // 8  # Due to 3 stride-2 operations (1 in stem, 2 in MBConv)
    w = input_shape[1] // 8
    sequence_length = h * w
    
    x = layers.Reshape((sequence_length, 384))(x)
    
    # Add positional embedding using custom layer
    x = PositionalEmbedding(sequence_length, 384)(x)
    
    # Transformer blocks
    x = transformer_block(x, 384)
    x = transformer_block(x, 384)
    
    # Classification head
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(num_classes, activation='softmax')(x)
    
    return keras.Model(inputs, x)



    # Create and compile model
model = create_proto_1()
model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

Found 140002 files belonging to 2 classes.
Found 39428 files belonging to 2 classes.
Found 10905 files belonging to 2 classes.


In [2]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 128, 128, 64)   │          1,792 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 128, 128, 64)   │            256 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 128, 128, 64)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 128, 128, 256)  │         16,384 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 128, 128, 256)  │          1,024 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 128, 128, 256)  │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ depthwise_conv2d          │ (None, 64, 64, 256)    │          2,304 │ activation_1[0][0]     │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 64, 64, 256)    │          1,024 │ depthwise_conv2d[0][0] │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 64, 64, 256)    │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d  │ (None, 256)            │              0 │ activation_2[0][0]     │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 64)             │         16,448 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 64)             │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 256)            │         16,640 │ activation_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape (Reshape)         │ (None, 1, 1, 256)      │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multiply (Multiply)       │ (None, 64, 64, 256)    │              0 │ activation_2[0][0],    │
│                           │                        │                │ reshape[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)    

 Total params: 5,049,410 (19.26 MB)

 Trainable params: 5,040,962 (19.23 MB)

 Non-trainable params: 8,448 (33.00 KB)

In [3]:
epochs = 15
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5
)

filepath = "/kaggle/working/model/"
modelName = "deepfake"
filepath = modelName + "_bestweights_256*256.keras"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint, early_stopping, reduce_lr]

# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=callbacks_list
)

Epoch 1/15
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step - accuracy: 0.7554 - loss: 0.5070
Epoch 1: val_accuracy improved from -inf to 0.70531, saving model to deepfake_bestweights_256*256.keras
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 1560s 349ms/step - accuracy: 0.7554 - loss: 0.5070 - val_accuracy: 0.7053 - val_loss: 0.6989 - learning_rate: 0.0010
Epoch 2/15
4375/4376 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step - accuracy: 0.9034 - loss: 0.2355
Epoch 2: val_accuracy improved from 0.70531 to 0.85959, saving model to deepfake_bestweights_256*256.keras
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 1511s 345ms/step - accuracy: 0.9034 - loss: 0.2355 - val_accuracy: 0.8596 - val_loss: 0.3527 - learning_rate: 0.0010
Epoch 3/15
4375/4376 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step - accuracy: 0.9123 - loss: 0.2133
Epoch 3: val_accuracy did not improve from 0.85959
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 1503s 344ms/step - accuracy: 0.9123 - loss: 0.2132 - val_accuracy: 0.6804 - val_loss: 0.6668 - learning_rate: 0.0010
Epoch 4/15
4375/4376 ━━━

In [4]:
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test accuracy: {test_accuracy:.4f}")

341/341 ━━━━━━━━━━━━━━━━━━━━ 37s 108ms/step - accuracy: 0.8023 - loss: 0.4643
Test accuracy: 0.7973


In [5]:
# Load the best saved model
loaded_model = keras.models.load_model(filepath)

TypeError: <class 'keras.src.models.functional.Functional'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'keras.src.models.functional', 'class_name': 'Functional', 'config': {'name': 'functional', 'trainable': True, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': [None, 256, 256, 3], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer'}, 'registered_name': None, 'name': 'input_layer', 'inbound_nodes': []}, {'module': 'keras.layers', 'class_name': 'Conv2D', 'config': {'name': 'conv2d', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'filters': 64, 'kernel_size': [3, 3], 'strides': [2, 2], 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': [1, 1], 'gr

In [ ]:
# Predictions
y_pred_probs = loaded_model.predict(test_ds)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.concatenate([y for x, y in test_ds], axis=0)
y_true_binary = np.argmax(y_true, axis=1)  # Convert to binary labels

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_true_binary, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Fake', 'Real'])
disp.plot()
plt.title('Confusion Matrix')
plt.savefig("/kaggle/working/model/confu.png")

In [ ]:
# Classification Report and Metrics
print("Classification Report:")
print(classification_report(y_true_binary, y_pred, target_names=['Fake', 'Real']))

f1 = f1_score(y_true_binary, y_pred)
precision = precision_score(y_true_binary, y_pred)
accuracy = accuracy_score(y_true_binary, y_pred)
recall = recall_score(y_true_binary, y_pred)

In [ ]:
print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")

In [ ]:
import matplotlib.pyplot as plt

fig2 = plt.figure(figsize=(10, 6))

plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.plot(history.history['accuracy'],  label='accuracy')

plt.plot(history.history['val_loss'], label='val_loss')
plt.plot(history.history['loss'],  label='loss')

plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Deepfake')
plt.legend()
plt.savefig('/kaggle/working/model/graph.png')
plt.show()